In [25]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
# import csvs
dblp = pd.DataFrame()
for i in range(1, 5):
    csv = pd.read_csv(f'../application/data/dblp-{i}.csv')
    dblp = pd.concat([dblp, csv]).reset_index(drop=True)

test_hidden = pd.read_csv('../application/data/test_hidden.csv')
train = pd.read_csv('../application/data/train.csv')
validation_hidden = pd.read_csv('../application/data/validation_hidden.csv')

## Making Predictions

We take out partition 8 to have a validation set with labels

In [27]:
train_data = train[train['partition']!=8]
val_data = train[train['partition']==8]

In [28]:

from googletrans import Translator
translator = Translator()
def translate(c):
  return translator.translate(c, dest='en').text

We swap the title and author when the title contains '|' or is too short

In [117]:
text_features = dblp[['pkey','pauthor', 'ptitle', 'pyear']]
text_features['pauthor'] = text_features['pauthor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
text_features['ptitle'] = text_features['ptitle'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
mask = text_features['ptitle'].str.contains('|', regex=False) | ((~text_features['pauthor'].str.contains('|', regex=False))
                                                    & (text_features['pauthor'].str.len() > text_features['ptitle'].str.len()))
text_features.loc[mask, ['ptitle', 'pauthor']] = text_features.loc[mask, ['pauthor','ptitle']].values
full_train = pd.merge(train, text_features, left_on='key1', right_on='pkey', suffixes=('', '_x'))
full_train = pd.merge(full_train, text_features, left_on='key2', right_on='pkey', suffixes=('_x', '_y'))


<ipython-input-117-fe3b86a95f4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_features['pauthor'] = text_features['pauthor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
<ipython-input-117-fe3b86a95f4c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_features['ptitle'] = text_features['ptitle'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()


We use Jacard similarity to compare the titles and authors

In [118]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.replace('|',' ').replace('-', ' ').split()) 
    b = set(str2.replace('|',' ').replace('-', ' ').split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))



In [119]:
full_train['similarity'] = full_train.apply(lambda x: get_jaccard_sim(x['pauthor_x'], x['pauthor_y']), axis=1)
full_train['similarity_title'] = full_train.apply(lambda x: get_jaccard_sim(x['ptitle_x'], x['ptitle_y']), axis=1)

In [120]:
full_train

,Unnamed: 0,key1,key2,label,partition,pkey_x,pauthor_x,ptitle_x,pyear_x,pkey_y,pauthor_y,ptitle_y,pyear_y,similarity,similarity_title
0,0,conf/prib/AhmedF07,journals/jcc/PatraS09,False,7,conf/prib/AhmedF07,said hassan ahmed|tor fla,estimation of evolutionary average hydrophobic...,2007,journals/jcc/PatraS09,jagdish chandra patra|onkar singh,artificial neural networks-based approach to d...,2009,0.000000,0.000000
1,1,conf/vlsid/ChenCC95,journals/tcad/LuoCWCCW08,True,4,conf/vlsid/ChenCC95,yung-yuan chen|ching-hwa cheng|jwu-e chen,an efficient switching network fault diagnosis...,1995,journals/tcad/LuoCWCCW08,pei-wen luo|jwu-e chen|chin-long wey|liang-chi...,impact of capacitance correlation on yield enh...,2008,0.263158,0.000000
2,2,conf/prozess/Sun88,conf/isnn/SunZLCS07,True,8,conf/prozess/Sun88,z. sun,anwendung graphischer darstellungen im rahmen ...,-1988,conf/isnn/SunZLCS07,z. sun|m. j. zhang|xiao h. liao|wenchuan cai|y...,neuro-adaptive formation control of multi-mobi...,-2007,0.166667,0.000000
3,3,conf/pricai/BeaumontTSM04,conf/icip/SattarAS08,False,5,conf/pricai/BeaumontTSM04,matthew beaumont|john thornton|abdul sattar|mi...,solving over-constrained temporal reasoning pr...,2004,conf/icip/SattarAS08,abdul sattar 0003|yasser aidarous|renaud seguier,gagm-aam: a genetic optimization with gaussian...,2008,0.142857,0.000000
4,774,conf/aiide/BourneS05,conf/icip/SattarAS08,False,1,conf/aiide/BourneS05,owen bourne|abdul sattar,applying constraint weighting to autonomous ca...,2005,conf/icip/SattarAS08,abdul sattar 0003|yasser aidarous|renaud seguier,gagm-aam: a genetic optimization with gaussian...,2008,0.222222,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7967,9993,journals/corr/abs-1011-1127,journals/corr/abs-1011-1119,True,4,journals/corr/abs-1011-1127,oleg chertov|dan tavrov,group anonymity: problems and solutions,2010,journals/corr/abs-1011-1119,oleg chertov|dan tavrov,group anonymity,2010,1.000000,0.166667
7968,9994,journals/corr/abs-cs-0607015,journals/isci/Valle-LisboaM07,True,5,journals/corr/abs-cs-0607015,juan c. valle-lisboa|eduardo mizraji,the uncovering of hidden structures by latent ...,2006,journals/isci/Valle-LisboaM07,juan c. valle-lisboa|eduardo mizraji,the uncovering of hidden structures by latent ...,2007,1.000000,0.800000
7969,9995,conf/splst/KalmanHG03,journals/infsof/KalmanHG06,True,7,conf/splst/KalmanHG03,miklos kalman|ferenc havasi|tibor gyimothy,compacting xml documents.,2003,journals/infsof/KalmanHG06,miklos kalman|ferenc havasi|tibor gyimothy,compacting xml documents.,2006,1.000000,1.000000
7970,9997,conf/esws/DavidK0RZZ10,journals/corr/abs-1004-3390,True,3,conf/esws/DavidK0RZZ10,catalin david|michael kohlhase|christoph lange...,publishing math lecture notes as linked data.,2010,journals/corr/abs-1004-3390,catalin david|michael kohlhase|christoph lange...,publishing math lecture notes as linked data,2010,1.000000,0.750000


In [125]:
prediction = full_train['similarity'] + full_train['similarity_title'] > 0.2

In [126]:
from sklearn.metrics import accuracy_score
half = int(len(prediction)/2)
accuracy_score(prediction, full_train['label'])

0.7521324636226794

In [107]:
full_train.iloc[7968]


Unnamed: 0                                                       9994
key1                                     journals/corr/abs-cs-0607015
key2                                    journals/isci/Valle-LisboaM07
label                                                            True
partition                                                           5
pkey_x                                   journals/corr/abs-cs-0607015
pauthor_x                        Juan C. Valle-Lisboa|Eduardo Mizraji
ptitle_x            The uncovering of hidden structures by Latent ...
pyear_x                                                          2006
pkey_y                                  journals/isci/Valle-LisboaM07
pauthor_y                        Juan C. Valle-Lisboa|Eduardo Mizraji
ptitle_y            The uncovering of hidden structures by Latent ...
pyear_y                                                          2007
similarity                                                        1.0
similarity_title    

In [12]:
text_features.head(50)

,pkey,pauthor,ptitle,pyear
0,conf/dft/SemiaoRVSTT07,Jorge Semião|Juan J. Rodríguez-Andina|Fabian V...,Improving the Tolerance of Pipeline Based Circ...,-2007
1,conf/dagstuhl/Caire07,Patrice Caire,A Normative Multi-Agent Systems Approach to th...,-2007
2,journals/tit/BT07,Sundeep B|Andrew Thangaraj,Self-Orthogonality of q-Ary Images of qm-Ary C...,2007
3,conf/icdcsw/Pardo-Castellote03,Gerardo Pardo-Castellote,OMG Data-Distribution Service: Architectural O...,-2003
4,journals/corr/abs-0911-4329,Ki-Hoon Lee|Kyu-Young Whang|Wook-Shin Han|Min-...,Structural Consistency: Enabling XML Keyword S...,2009
5,conf/cdc/LiuHT09,Bin Liu|David J. Hill|Kok Lay Teo,Input-to-state stability for a class of hybrid...,-2009
6,journals/mcs/SunKZC03,Haiwei Sun|Ning Kang|Jun Zhang|Eric S. Carlson,A fourth-order compact difference scheme on fa...,-2003
7,conf/icic/LeeKKK06,Sang-Heon Lee|Nam-Sung Kim|Heau-Jo Kang|Soon-G...,Performance Improvement of Intelligent UWB-IR ...,-2006
8,conf/icns/Capo-ChichiGF09,Eugene Pamba Capo-Chichi|Hervé Guyennet|Jean-M...,IEEE 802.15.4 Performance on a Hierarchical Hy...,-2009
9,journals/jche/WangH09,Feng Wang|Michael J. Hannafin,Scaffolding preservice teachers' WebQuest desi...,-2009


In [13]:
dblp[['pauthor', 'ptitle']].head(20)

,pauthor,ptitle
0,Jorge Semião|Juan J. Rodríguez-Andina|Fabian V...,Improving the Tolerance of Pipeline Based Circ...
1,Patrice Caire,A Normative Multi-Agent Systems Approach to th...
2,Sundeep B|Andrew Thangaraj,Self-Orthogonality of q-Ary Images of qm-Ary C...
3,Gerardo Pardo-Castellote,OMG Data-Distribution Service: Architectural O...
4,Ki-Hoon Lee|Kyu-Young Whang|Wook-Shin Han|Min-...,Structural Consistency: Enabling XML Keyword S...
5,Bin Liu|David J. Hill|Kok Lay Teo,Input-to-state stability for a class of hybrid...
6,A fourth-order compact difference scheme on fa...,Haiwei Sun|Ning Kang|Jun Zhang|Eric S. Carlson
7,Sang-Heon Lee|Nam-Sung Kim|Heau-Jo Kang|Soon-G...,Performance Improvement of Intelligent UWB-IR ...
8,Eugene Pamba Capo-Chichi|Hervé Guyennet|Jean-M...,IEEE 802.15.4 Performance on a Hierarchical Hy...
9,Feng Wang|Michael J. Hannafin,Scaffolding preservice teachers' WebQuest desi...


In [136]:
db = pd.read_csv('data/db/db.csv', sep='!')

In [137]:
db.clean_author = db.clean_author.str.replace('-', ' ').str.replace('|', ' ')
db['clean_author']= db['clean_author'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')#.str.lower()
db['clean_title']= db['clean_title'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

<ipython-input-137-a50a713c55a2>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  db.clean_author = db.clean_author.str.replace('-', ' ').str.replace('|', ' ')


In [138]:
db.set_index('_c0').to_csv('data/db/db2.csv', sep='!')

In [134]:
val = pd.read_csv('data/validation_hidden.csv')

In [17]:
val[val['key1']]

KeyError: "None of [Index(['conf/icdim/KebailiA07', 'conf/spaa/AndersonBCKSSSTWZ05',\n       'journals/comgeo/MustafaR10', 'journals/tog/RenWSZLSSBPG06',\n       'conf/lctrts/TanM05', 'conf/webi/RamanujamGKST09', 'series/asc/Kim07',\n       'conf/advis/KimKYKO04', 'journals/tip/JiangJM06',\n       'journals/jcisd/MattioniKJCDP03',\n       ...\n       'journals/ai/KellyP10', 'conf/hicss/HainesL04', 'journals/toms/Hill81c',\n       'conf/vlsid/ZhouLW07', 'conf/csreaESA/WarrenAWH08', 'conf/csse/CuiWC08',\n       'conf/micro/EssinkADGKV91', 'conf/agp/PuigsegurA98',\n       'journals/vldb/MehtaGPN98', 'journals/toms/Hill81b'],\n      dtype='object', length=994)] are in the [columns]"

In [135]:
db

,_c0,peditor,pyear,paddress,ppublisher,pseries,pid,pkey,ptype_id,pjournal_id,pbooktitle_id,pjournalfull_id,pbooktitlefull_id,partition,clean_author,clean_title
0,4,NaN,2007.0,NaN,NaN,NaN,180843.0,conf/dft/SemiaoRVSTT07,1,0.0,4,0.0,4.0,1.0,jorge semiao juan j. rodriguez andina fabian v...,improving the tolerance of pipeline based circ...
1,7,NaN,2007.0,NaN,NaN,NaN,162991.0,conf/dagstuhl/Caire07,2,0.0,7,0.0,7.0,1.0,patrice caire,a normative multi-agent systems approach to th...
2,10,NaN,2007.0,NaN,NaN,NaN,2261406.0,journals/tit/BT07,0,2.0,9,2.0,9.0,1.0,sundeep b andrew thangaraj,self-orthogonality of q-ary images of qm-ary c...
3,18,NaN,2003.0,NaN,NaN,NaN,349720.0,conf/icdcsw/Pardo-Castellote03,1,0.0,11,0.0,11.0,1.0,gerardo pardo castellote,omg data-distribution service: architectural o...
4,19,NaN,2009.0,NaN,NaN,NaN,1922328.0,journals/corr/abs-0911-4329,3,5.0,5,5.0,5.0,1.0,ki hoon lee kyu young whang wook shin han min ...,structural consistency: enabling xml keyword s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17160,9975,NaN,2008.0,NaN,NaN,NaN,2245299.0,journals/tcs/ShiY08,0,12.0,5,12.0,5.0,4.0,yongqiang shi deshi ye,online bin packing with arbitrary release times.
17161,9978,NaN,2001.0,NaN,NaN,NaN,460233.0,conf/ijcai/TanC01,2,0.0,71,0.0,71.0,4.0,chew lim tan henry wai kit chia,neural logic network learning using genetic pr...
17162,9981,NaN,2004.0,NaN,NaN,NaN,289610.0,conf/hicss/NewellSW04,1,0.0,295,0.0,295.0,4.0,sue newell jacky swan joseph weiss,project management: minitrack introduction.
17163,9988,NaN,2006.0,NaN,NaN,NaN,2224414.0,journals/sttt/HendriksNV06,3,374.0,5,374.0,5.0,4.0,martijn hendriks barend van den nieuwelaar fri...,model checker aided design of a controller for...
